In [1]:
import re
import os
from tqdm import tqdm

In [2]:
def slurp(path):
    try:
        with open(path, 'r') as fo:
            text = fo.read()
    except UnicodeDecodeError:
        print(path)
        with open(path, 'r', encoding='cp1252') as fo:
            text = fo.read()
    return text

def spit(texts, file_names):
    for text, file_name in tqdm(zip(texts, file_names)):
        with open(file_name, 'w') as fo:
            fo.write(text)

def read_dir(input_path):
    texts = []
    files = []
    print('Reading files...')
    for root, dirs, filenames in os.walk(input_path):
        files.extend(filenames)
        for filename in tqdm(filenames):
            file_path = os.path.join(root, filename)
            if '.ipynb' not in file_path:
                text = slurp(file_path)
                texts.append(text)
    print('Number of texts: ', len(texts))
    return texts, files

def preprocess(input_path, output_path):
    texts, filenames = read_dir(input_path)
    pattern = re.compile(r'[А-ЯЁа-яё\.\-\d]+')
    preprocessed = []
    print('Preprocessing files...')
    for text in tqdm(texts):
        preproc_text = ' '.join(re.findall(pattern, text))
        preprocessed.append(preproc_text)
    paths = [output_path + name for name in filenames if 'ipynb' not in name]
    print('Number of texts: ', len(texts))
    print('Number of paths:', len(paths))
    print('Writing to files...')
    spit(preprocessed, paths)
    print('All done, Buddy!')

## Preprocess texts before parsing

In [81]:
spit_dir = '/home/nst/mount/data/linguistics_hse/popular-science-research/Tomita_Parser/tomita-parser/build/bin/sci_corpus/'
slurp_dir = '/home/nst/mount/data/share/yd/popular_science_texts_store_copy'

In [92]:
chrdk = preprocess(slurp_dir, spit_dir)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/707 [00:00<?, ?it/s]

Reading files...



 17%|█▋        | 117/707 [00:01<00:07, 74.23it/s]Exception in thread Thread-131:
Traceback (most recent call last):
  File "/home/nst/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/nst/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/nst/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 5166/5166 [00:58<00:00, 87.77it/s]
0it [00:00, ?it/s]
100%|██████████| 1454/1454 [00:15<00:00, 96.60it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 2452.81it/s]
0it [00:00, ?it/s]
100%|██████████| 3850/3850 [00:58<00:00, 65.50it/s]
0it [00:00, ?it/s]
  0%|          | 67/31052 [00:00<00:46, 668.37it/s]

Number of texts:  31052
Preprocessing files...


100%|██████████| 31052/31052 [00:13<00:00, 2247.23it/s]
0it [00:00, ?it/s]

Number of texts:  31052
Number of paths: 31052
Writing to files...


31052it [06:37, 78.14it/s] 


All done, Buddy!


## Launch on test sample 

In [3]:
test_input = '/home/nst/mount/data/share/yd/popular_science_texts_store/ner_markup/final_markup/'

In [5]:
marked = []
for root, dirs, filenames in os.walk(test_input):
    for filename in tqdm(filenames):
        path = test_input + filename
        text = slurp(path)
        marked.append(text)
marked = list(set(marked))

100%|██████████| 175/175 [00:00<00:00, 1236.56it/s]


In [87]:
for text in marked:
    if 'Марком Ависом' in text:
        print(marked.index(text))
        
marked.pop(166)

90
166


'Крысы в штанах, независимые камни и человек-козел стали героями двадцать шестой Шнобелевской премии. О том, какие еще исследования были отмечены наградой, рассказывает Indicator.Ru.\nМеждународная премия за научные достижения обошла вниманием российских исследователей. &Может!&, это и к лучшему, ведь речь идет о «Шнобелевке». В театре Сандерс при Гарвардском университете прошла церемония вручения двадцать шестой Шнобелевской премии – 26th Ig Nobel Prize. Название премии построено на игре слов: "ignoble" переводится с английского языка как «позорный». «Антинобелевскую» премию ежегодно вручают за научные исследования, которые «заставляют сначала посмеяться, а после задуматься».\nКаждый год «Шнобеля» вручают нобелевские лауреаты. В этот раз организаторы пригласили &Дадли Хершбаха!& (химия, 1986), &Эрика Маскина!& (экономика, 2007), &Ричарда Робертса!& (физиология и медицина, 1993) и &Роя Глаубера!& (физика, 2005). \nЧтобы получить награды за сомнительные научные достижения, лауреатам при

### Find names in marked text

In [88]:
pattern = re.compile(r'\&[А-ЯЁа-яё\-\s]+!\&')
names_list = []
for text in marked:
    names = re.findall(pattern, text)
    clean_names = [name.replace('&', '').replace('!', '') for name in names]
    names_list.extend(clean_names)

In [93]:
to_pop = [51, 52, 54, 123 , 131, 151, 157, 185, 196, 381, 398, 461, 511, 600, 655, 664, 929,
          982, 1210, 1236, 1261, 1334, 1340, 1401, 1408, 1503, 1511, 1527, 1582, 1638, 1656,
         1670, 1704, 1706]
for i in to_pop:
    names_list.pop(i)

In [97]:
with open('evaluation_names.txt', 'w') as fo:
    for name in names_list:
        fo.write(name+'\n')

## Test sample texts

In [99]:
marked_texts, file_names = read_dir(test_input)
texts = [text.replace('&', '').replace('!', '') for text in marked_texts]

100%|██████████| 175/175 [00:00<00:00, 1619.14it/s]

Reading files...
Number of texts:  175


In [105]:
texts.pop(8)
texts.pop(166)

'Выдающие специалисты, обладающие бесспорным опытом в научных областях, поделились с читателями Geektimes своими короткими комментариями относительно перспектив различных профессий в области медицины.\nИТ медик\n\nМажуга Александр Георгиевич\nПодробнее о научной деятельности\nПрофессия будет востребована, но в перспективе лет. \nПрогресс в области современной медицинской химии и фармакологии обеспечивает создание новых классов препаратов для терапии патологий различной этиологии. \nОбразование базового врача общего профиля не позволяет адекватно оценивать совместимость лекарственных препаратов и возможные эффекты связанные с их применением. Задача ИТ медика разработать информационное «облако», которое будет автоматизировано оценивать риски приема лекарственных препаратов. \nДругое направление – это обслуживание диагностических комплексов, разработка методологий оценки результатов исследований и рекомендации по терапии, это и есть задача будущей ИТ медицины.\nБиоэтик\n\nЧрезвычайно важн

In [107]:
file_names.pop(8)
file_names.pop(166) ;

In [102]:
test_output = '/home/nst/mount/data/linguistics_hse/popular-science-research/Tomita_Parser/tomita-parser/build/bin/sci_names_eval/'

In [109]:
pattern = re.compile(r'[А-ЯЁа-яё\.\-\d]+')
preprocessed = []
print('Preprocessing files...')
for text in tqdm(texts):
    preproc_text = ' '.join(re.findall(pattern, text))
    preprocessed.append(preproc_text)
paths = [test_output + name for name in file_names]
print('Number of texts: ', len(texts))
print('Number of paths:', len(paths))
print('Writing to files...')
spit(preprocessed, paths)
print('All done, Buddy!')

 53%|█████▎    | 92/173 [00:00<00:00, 910.68it/s]

Preprocessing files...


100%|██████████| 173/173 [00:00<00:00, 848.19it/s]
173it [00:00, 1218.56it/s]

Number of texts:  173
Number of paths: 173
Writing to files...
All done, Buddy!


# NER-PARSER

In [3]:
import bs4 as bs

In [24]:
output = slurp('/home/nst/mount/data/linguistics_hse/popular-science-research/Tomita_Parser/tomita-parser/build/bin/evaluation.xml')

In [25]:
def parse_xml(xml_output):
    
    def divide_names(names:list):
        names_found = []
        for name in names:
            if len(name.split()) > 3:
                n_sep = name.split()
                start_index = 0
                for n in n_sep:
                    while start_index != len(n_sep):
                        name = n_sep[start_index] + ' ' + n_sep[start_index+1]
                        start_index+=2
                        names_found.append(name)
            else:
                names_found.append(name)
        return names_found
    
    
    xml = bs.BeautifulSoup(xml_output, 'lxml')
    names = xml.find_all('name')
    pattern = re.compile(r'val="([А-ЯЁ]+)">')
    names_ext = []
    for name in names:
        name = name.get('val')
        names_ext.append(name)
    names_ext = [name.title() for name in names_ext]
    names_sep = divide_names(names_ext)
    return names_sep

In [26]:
names = parse_xml(output)
len(names)

683

In [27]:
true = slurp('/home/nst/mount/data/linguistics_hse/popular-science-research/Tomita_Parser/tomita-parser/build/bin/evaluation_names.txt')
true = true.split('\n')

In [28]:
pred = []
for name in names:
    if name in true:
        pred.append(name)
accuracy = len(pred)/len(true)
print(accuracy)

0.15156695156695157


In [29]:
len(true)

1755

In [30]:
len(pred)

266

## Delete common words and geoterms

In [31]:
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()
from flashtext import KeywordProcessor
from tqdm import tqdm
import pandas as pd

In [32]:
def compile_lemmas_list(input_path):
    lemmas_list = []
    for root, dirs, files in os.walk(input_path):
        for file_name in files:
            file_path = input_path + file_name
            lemmas = slurp(file_path)
            lemmas = re.findall(r'[а-яё]+', lemmas)
            lemmas_list.extend(lemmas)
    lemmas_list = list(set(lemmas_list))
    return lemmas_list

In [33]:
common = '/home/nst/mount/data/linguistics_hse/popular-science-research/slovnik/'
common_words = compile_lemmas_list(common)

In [34]:
lemmas_processor = KeywordProcessor()
lemmas_processor.add_keywords_from_list(common_words)

In [35]:
cities = pd.read_csv('/home/nst/mount/data/linguistics_hse/popular-science-research/cities.csv', sep='\t')
countries = pd.read_csv('/home/nst/mount/data/linguistics_hse/popular-science-research/countries.csv', sep='\t')
geo = cities.москва + countries.россия

In [36]:
lemmas_geo = KeywordProcessor()

In [37]:
def delete_common_words(names:list):
    potential_names = []
    
    for name in names:
        true_name = []
        name = name.split()
        for n in name:
            n = n.lower()
            lemma = morph.parse(n)[0].normal_form
            if not lemmas_processor.extract_keywords(lemma) == [lemma]:
                true_name.append(n.title())
                final_name = ' '.join(true_name)
                potential_names.append(final_name)
    print('Deleted common words, current size:', len(potential_names))
    return potential_names

In [18]:
def count_accuracy(true:list, eval_list:list):
    pred = []
    for name in eval_list:
        if name in true:
            pred.append(name)
    accuracy = len(pred)/len(true)
    print('Accuracy:', accuracy)

In [38]:
names_clean = delete_common_words(names)
count_accuracy(true, names_clean)

Deleted common words, current size: 630
Accuracy: 0.1492877492877493
